### Mapping state-level changes to real GDP

Data source: U.S. Bureau of Economic Analysis

https://www.bea.gov/newsreleases/regional/gdp_state/qgsp_newsrelease.htm



In [1]:
import requests
import pandas as pd
import config   ## File with API key
api_key = config.key

# Components of request
base = 'https://www.bea.gov/api/data/?&UserID={}'.format(api_key)
m = '&method=GetData'
dsn = '&datasetname=RegionalProduct'
ind = '&IndustryId=1'
comp = '&Component=RGDP_SQN'
geo = '&GeoFIPS=STATE'
year = '&Year=LAST5'
fmt = '&ResultFormat=json'

# Combined url for request
url = '{}{}{}{}{}{}{}{}'.format(base, m, dsn, ind, comp, geo, year, fmt)

# Load list of state names and postal abbreviations
states = pd.read_csv('state_abbrevs.csv').set_index('State')

In [2]:
# Make request and load data into pandas dataframe, show last 5 obs
r = requests.get(url).json()
df = pd.DataFrame(r['BEAAPI']['Results']['Data'])
df = df[df['GeoName'].isin(states.index)]
df['date'] = pd.to_datetime(df['TimePeriod'])
df = df.sort_values(['GeoName', 'date'])
df = df.set_index(['GeoName', 'date'])['DataValue']
df = df.apply(pd.to_numeric, errors='coerce').unstack(0)

In [3]:
# Calculate growth rate by state
for s in df.keys():
    df[s] = ((1 + (df[s] - df[s].shift(1))/df[s].shift(1))**4 - 1) * 100

In [4]:
date = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
df1 = df.tail(1).stack().reset_index()[['GeoName', 0]]
df1['Abbrev'] = df1['GeoName'].map(states.to_dict()['Abbreviation'])
df1 = df1.set_index('GeoName').set_value('Michigan', 'Abbrev', 'MI, SP')

#### Convert to .tex input



In [5]:
# Tex strings 
c_str = '\\tikzset{set state val/.style args={#1}{#1='
s_str = '\\tikzset{set state val/.list={'

# Color categories
C1 = '{fill=blue!50!green}}}'
C2 = '{fill=green!80!blue}}}'
C3 = '{fill=yellow!70!green}}}'
C4 = '{fill=orange!60!yellow}}}'
C5 = '{fill=red!40!orange!80}}}'
C6 = '{fill=red!80!black!80}}}'

In [6]:
S1 = ', '.join(map(str, df1[df1[0] >= 5]['Abbrev'].values))
S2 = ', '.join(map(str, df1[(df1[0] < 5) & (df1[0] >= 2)]['Abbrev'].values))
S3 = ', '.join(map(str, df1[(df1[0] < 2) & (df1[0] >= 0)]['Abbrev'].values))
S4 = ', '.join(map(str, df1[(df1[0] < 0) & (df1[0] >= -2)]['Abbrev'].values))
S5 = ', '.join(map(str, df1[(df1[0] < -2) & (df1[0] >= -5)]['Abbrev'].values))
S6 = ', '.join(map(str, df1[df1[0] < -5]['Abbrev'].values))

In [7]:
d = {C1: S1, C2: S2, C3: S3, C4: S4, C5: S5, C6: S6}
for key, value in d.iteritems():
    print '{}{}\n{}{}}}}}'.format(c_str, key, s_str, value)

\tikzset{set state val/.style args={#1}{#1={fill=red!80!black!80}}}
\tikzset{set state val/.list={}}
\tikzset{set state val/.style args={#1}{#1={fill=yellow!70!green}}}
\tikzset{set state val/.list={AL, AK, AZ, AR, CA, CO, CT, DE, DC, FL, GA, IL, IN, KY, LA, ME, MA, MI, SP, MS, MO, NV, NH, NJ, NY, NC, ND, OH, OK, OR, PA, RI, SC, TN, UT, VT, WY}}
\tikzset{set state val/.style args={#1}{#1={fill=green!80!blue}}}
\tikzset{set state val/.list={ID, MD, NM, TX, VA, WA, WV, WI}}
\tikzset{set state val/.style args={#1}{#1={fill=red!40!orange!80}}}
\tikzset{set state val/.list={IA, NE, SD}}
\tikzset{set state val/.style args={#1}{#1={fill=blue!50!green}}}
\tikzset{set state val/.list={}}
\tikzset{set state val/.style args={#1}{#1={fill=orange!60!yellow}}}
\tikzset{set state val/.list={HI, KS, MN, MT}}


In [8]:
df1.sort_values(0)

,0,Abbrev
GeoName,,
Nebraska,-3.954941,NE
South Dakota,-3.847546,SD
Iowa,-3.245099,IA
Hawaii,-0.871814,HI
Kansas,-0.745055,KS
Montana,-0.523204,MT
Minnesota,-0.330072,MN
Maine,0.030776,ME
California,0.098929,CA


In [9]:
date

'2017 Q1'